In [1]:
#현재 수준에서 진행이 안되는 것으로 확인, diff(닭고기 소고기 쇠고기 구분 못함)나 kolnpy 수준에서 단순 대조만 가능 

# Preprocessing

In [2]:
# !pip install transformers
# !pip install accelerate
# !pip install soynlp
import pandas as pd
import re
df_allergy = pd.read_csv('0515.csv')
df_allergy

,Unnamed: 0,rnum,rawmtrl,allergy
0,0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음
1,1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드
2,2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음
3,3,4,군고구마,없음
4,4,5,군고구마,없음
...,...,...,...,...
15087,15087,15088,"된장63.15%[대두{수입산(미국산,중국산,캐나다산)},소맥분(밀:미국산,호주산),...","대두,밀"
15088,15088,15089,"물엿,소맥분(밀:미국산,호주산),혼합양념(중국산)[고춧가루6.12%,(고추:중국산)...",밀
15089,15089,15090,"탈지대두[수입산(인도산,미국산,중국산)],정제수,양조간장원액(대만산)[탈지대두,소맥...","대두,밀"
15090,15090,15091,"정제수,아미노산액[탈지대두:외국산(인도산,미국산,중국산등)],정제소금(국산),액상과...","대두,밀"


In [3]:
from soynlp.normalizer import *
import re

def preprocessing(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])
        
    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])
    
    # ~산 제거 (아황산 제외)
    text = re.sub(r'\b(?!아황산)\w+산\b', '', text).strip()

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()
    
    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()
    
    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()
    
    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()
    
    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]
    
    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()
    
    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()
    
    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()
    
    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    
    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()
    
    # 개행 문자 "\n" 제거
    text = text.replace('\n', '')
    
    # 영어 제거
    text = re.sub(r'[a-zA-Z]', '', text)
    return text

In [4]:
# def remove_useless_breacket_l(texts):
#     """
#     위키피디아 전처리를 위한 함수입니다.
#     괄호 내부에 의미가 없는 정보를 제거합니다.
#     아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
#     ``수학(,)`` -> ``수학``
#     ``수학(數學,) -> ``수학(數學)``
#     """
#     bracket_pattern = re.compile(r"\((.*?)\)")
#     preprocessed_text = []
#     for text in texts:
#         modi_text = ""
#         text = text.replace("()", "")  # 수학() -> 수학
#         brackets = bracket_pattern.search(text)
#         if not brackets:
#             if text:
#                 preprocessed_text.append(text)
#                 continue
#         replace_brackets = {}
#         # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
#         # e.g. {'2,8': '(數學)','34,37': ''}
#         while brackets:
#             index_key = str(brackets.start()) + "," + str(brackets.end())
#             bracket = text[brackets.start() + 1 : brackets.end() - 1]
#             infos = bracket.split(",")
#             modi_infos = []
#             for info in infos:
#                 info = info.strip()
#                 if len(info) > 0:
#                     modi_infos.append(info)
#             if len(modi_infos) > 0:
#                 replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
#             else:
#                 replace_brackets[index_key] = ""
#             brackets = bracket_pattern.search(text, brackets.start() + 1)
#         end_index = 0
#         for index_key in replace_brackets.keys():
#             start_index = int(index_key.split(",")[0])
#             modi_text += text[end_index:start_index]
#             modi_text += replace_brackets[index_key]
#             end_index = int(index_key.split(",")[1])
#         modi_text += text[end_index:]
#         modi_text = modi_text.strip()
#         if modi_text:
#             preprocessed_text.append(modi_text)
#     return preprocessed_text


In [5]:
import re

def remove_useless_breacket(text):
    """
    위키피디아 전처리를 위한 함수입니다.
    괄호 내부에 의미가 없는 정보를 제거합니다.
    아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
    ``수학(,)`` -> ``수학``
    ``수학(數學,) -> ``수학(數學)``
    """
    bracket_pattern = re.compile(r"\((.*?)\)|\[(.*?)\]|\{(.*?)\}")
    modi_text = ""
    text = text.replace("()", "")  # 수학() -> 수학
    brackets = bracket_pattern.search(text)
    if not brackets:
        if text:
            return text
    replace_brackets = {}
    # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
    # e.g. {'2,8': '(數學)','34,37': ''}
    while brackets:
        index_key = str(brackets.start()) + "," + str(brackets.end())
        bracket = brackets.group(0)[1:-1]
        infos = bracket.split(",")
        modi_infos = []
        for info in infos:
            info = info.strip()
            if len(info) > 0:
                modi_infos.append(info)
        if len(modi_infos) > 0:
            replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
        else:
            replace_brackets[index_key] = ""
        brackets = bracket_pattern.search(text, brackets.start() + 1)
    end_index = 0
    for index_key in replace_brackets.keys():
        start_index = int(index_key.split(",")[0])
        modi_text += text[end_index:start_index]
        if replace_brackets[index_key] and (not modi_text or modi_text[-1] != ","):
            modi_text += ","
        modi_text += replace_brackets[index_key]
        end_index = int(index_key.split(",")[1])
    modi_text += text[end_index:]
    modi_text = modi_text.strip()
    return modi_text

# 텍스트 스트링을 함수에 적용하여 수정
text = "파이썬(,)은(는) [데이터 분석]{data analysis} {머신 러닝}(machine learning) {} [] 프로그래밍 언어입니다."
modified_text = remove_useless_breacket(text)
print(modified_text)


파이썬은,(는) ,(데이터 분석),(data analysis) ,(머신 러닝),(machine learning)   프로그래밍 언어입니다.


In [6]:
def remove_bad_char(texts):
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    preprocessed_text  = []
    for text in texts:
        for bad_char in bad_chars:
            text = text.replace(bad_char, bad_chars[bad_char])
        text = re.sub(r"[+á?\xc3\xa1]", "", text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [7]:
def remove_email(texts):
    preprocessed_text  = []
    for text in texts:
        text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [8]:
def remove_numbers(text):
    # 정규표현식을 사용하여 숫자를 제거합니다
    cleaned_text = re.sub(r'\d+', '', text)
    return cleaned_text

In [9]:
def remove_hashtag(texts):
    preprocessed_text  = []
    for text in texts:
        text = re.sub(r"#\S+", "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [10]:
def clean_punc(texts):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    preprocessed_text  = []
    for text in texts:
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        text = text.strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [11]:
def clean_text(text):
    # 문자열에서 %, /, \, :를 제거합니다
    cleaned_text = text.replace('<', '').replace('>', '').replace('【', '').replace('】', '').replace('^', '').replace('%', '').replace('/', '').replace('\\', '').replace(':', '').replace(';', '').replace('*', '').replace('-', '').replace('등급', '').replace('이상', '').replace('미국', '').replace('호주', '').replace('베트남', '').replace('프랑스', '').replace('독일', '').replace('아르헨티나', '').replace('일본', '').replace('중국', '').replace('캐나다', '').replace('러시아', '').replace('말레이시아', '').replace('칠레', '').replace('브라질', '').replace('인도네시아', '').replace('인도', '').replace('담양', '').replace('소고기', '쇠고기').replace('오스트리아', '').replace('터키', '').replace('콜롬비아', '').replace('º', '').replace('세균수기준', '').replace('우크라이나',"").replace("싱가포르","").replace("네덜란드","")
    # 문자열에서 .을 ,로 바꿉니다
    cleaned_text = cleaned_text.replace('.', ',')
    return cleaned_text

In [12]:
def add_spaces(text):
    # 정규표현식을 사용하여 특수문자 앞뒤에 공백을 추가합니다
    cleaned_text = re.sub(r'([,.\(\)\[\]{}/\\:;%*])', r' \1 ', text)
    return cleaned_text

In [13]:
import re

def replace_brackets(text):
    # 괄호 및 문장 내 공백을 , 로 변경
    text = re.sub(r"[()\[\]{}]", ",", text)
    # 문장 내 공백 제거
    text = text.replace(" ", "")
    # 중복된 , 를 1개로 변경
    text = re.sub(r",+", ", ", text)
        
    return text

# 텍스트 스트링을 함수에 적용하여 수정
text = "파이썬(,)은(는) [데이터 분석]{data analysis} {머신 러닝}(machine learning) 프로그래밍 언어입니다."
modified_text = replace_brackets(text)
print(modified_text)

파이썬, 은, 는, 데이터분석, dataanalysis, 머신러닝, machinelearning, 프로그래밍언어입니다.


In [14]:
def remove_duplicates(text):
    # 문장을 단어로 분할하여 리스트로 저장
    words = text.split(", ")
    
    # 중복된 단어를 제거하여 새로운 리스트 생성
    unique_words = list(set(words))
    
    # 단어들을 다시 ', '로 구분하여 하나의 문자열로 반환
    modified_text = ", ".join(unique_words)

    # 문장의 제일 앞에 있는 ', '를 제거
    modified_text = modified_text.lstrip(", ")
    
    return modified_text

# 텍스트 스트링을 함수에 적용하여 수정
text = "apple, banana, cherry, apple, durian, cherry"
modified_text = remove_duplicates(text)
print(modified_text)

apple, banana, cherry, durian


In [15]:
import re

def remove_words_with_suffix(text, suffix):
    pattern = r'\b\w+' + suffix + r'\b'
    result = re.sub(pattern, '', text)
    return result.strip()

# 예시 텍스트
text = "사과, 배, 딸기, 바나나, 오렌지, 수박등, 복숭아등, 포도, 체리등"

# '등'으로 끝나는 단어 제거 적용
modified_text = remove_words_with_suffix(text, "등")

print(modified_text)

사과, 배, 딸기, 바나나, 오렌지, , , 포도,


In [16]:

def remove_pattern_words(text):
    patterns = ['국가', '홈페이지', '별도', '표기','※','특정','성분','성분명','및','함량']
    result = []
    
    words = text.split(',')
    for word in words:
        word = word.strip()  # 단어 앞뒤 공백 제거
        
        # 패턴 단어 포함 여부 확인
        if not any(pattern in word for pattern in patterns):
            result.append(word)
    
    result_text = ', '.join(result)
    return result_text

In [17]:
#df_allergy['rawmtrl_list'] = 
df_allergy['rawmtrl'][0]

'찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수'

In [18]:
df_allergy['preprocessing'] = df_allergy['rawmtrl'].apply(add_spaces).apply(remove_numbers).apply(remove_pattern_words).apply(preprocessing).apply(clean_text).apply(lambda x: remove_words_with_suffix(x, '등')).apply(remove_useless_breacket).apply(replace_brackets).apply(remove_duplicates).apply(replace_brackets)


In [19]:
df_allergy.sample(10) #12547
#df_allergy['preprocessing'][13907]#

,Unnamed: 0,rnum,rawmtrl,allergy,preprocessing
9673,9673,9674,"콩(국산)55%, 정제수, 천일염(국산), 종국(국산), 주정",알수없음,"콩, 종국, 정제수, 천일염, 주정"
11733,11733,11734,"생크림36.28%(국산),정제수,멸균우유(국산),복합시즈닝[혼합탈지분유(네덜란드산)...",알수없음,"말토덱스트린, 옥수수유, 흑후추분태, 변성전분, 체다치즈파우더, 아미노베이스, 잔탄..."
10897,10897,10898,"밀가루(미국산),식물성유지[팜올레인유(말레이시아산),팜핵경화유(말레이시아산),팜스테...","밀,대두,계란,우유","팜올레인유, 딸기분말, 바닐라향분말, 팜스테아린유, 유청단백분말, 딸기향, 밀가루,..."
3492,3492,3493,"유기농발아현미98.7%,천일염0.8%,스테비아0.2%,유기농이온화강화칼슘쌀눈0.3%",알수없음,"스테비아, 유기농이온화강화칼슘쌀눈, 천일염, 유기농발아현미"
12645,12645,12646,"면:쌀면(베트남산:쌀가루87%, 변성전분, 백설탕, 정제소금, 구아검)\n스프류:프...","밀, 대두, 계란, 우유, 쇠고기, 돼지고기, 닭고기, 오징어, 조개류","변성전분, 목심, 건고추입자, 팬더굴소스, 정제소금, 쌀가루, 건청경채, 구아검, ..."
10979,10979,10980,"발효식초[주정,레몬농축액(레몬:이스라엘산),혼합제제{주정,레몬향(합성착향료),초산리...","대두,밀","양조간장원액, 진한참기름, 레몬농축액, 레몬, 초산리나릴, 파인애플, 정제소금, 밀..."
11419,11419,11420,"돼지고기(삼겹살,수입산),정제수,난백분말(프랑스산-계란),D-소르비톨액,산도조절제,...",알수없음,"천연첨가물, 복합허브분말, 정제소금, 우유, 스모크오일, 글루타민산나트륨, 코치닐추..."
13714,13714,13715,"정제수,액상과당,백설탕,탄산가스,구연산,혼합제제(정제가공유지,유화제,D-소르비톨액,...",알수없음,"탄산가스, 유화제, 정제가공유지, 백설탕, 글리세린, 메타인산나트륨, 합성감미료, ..."
8392,8392,8393,"정제수, 당시럽, 이산화탄소, 구연산, 구연산삼나트륨, 천연향료, 합성향료(레몬라임...",알수없음,"합성향료, 정제수, 이산화탄소, 당시럽, 레몬라임향, 프로필렌글리콜, 천연향료, 구..."
14787,14787,14788,"면/소맥분(호주산),감자전분(독일산),팜유(말레이시아산),변성전분,식물성풍미유,난각...","밀, 대두, 계란, 우유, 게, 새우, 돼지고기, 쇠고기, 오징어, 조개류","변성전분, 건다시마, 면류첨가알칼리제, 매운양념분말, 해물채소분말, 글루텐펩타이드,..."


# 라벨링

In [20]:
# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','추리알','난각','날류','난백','난황','난각','게란','전란','흰자','노른자'], 
                2: ['우유','유우', '탈지분유','유유','분유','유청','카제인','우류','가공버터','유크림','크림','유청','치즈','원유', '요구르트', '요거트','유당','아이스크림','연유','농축유' ], 
                3: ['메밀'], 
                4: ['땅콩', '탕콩', '피넛'],
                5: ['대두','두부','콩','유부','된장','대두유','대두단백','레시틴'], 
                6: ['밀','밀가루','소맥','소맥분', '밀제품'], 
                7: ['고등어'], 
                8: ['게'], 
                9: ['새우'], 
                10: ['돼지','돈','돼지고기','괘지','괘지고기','돼고기','돈창', '돈혈', '돈지방','괘','젤라틴','돈육','순대'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토', '토마투','토마도','토마','토미토'],
                13: ['아황산', '아황산나트륨', '이황산', '이산화', '아산화', '이산화황','아산화황','이산화항','이황산류'],
                14: ['호두','호도','월넛'], 
                15: ['닭', '닭고기', '계육','닭괴','치킨','닭다리', '닭정육', '계정육', '닭가슴살', '닭안심'], 
                16: ['쇠고기', '우육', '소고기', '소뼈','쇠구기', '한우', '우정육', '소정육','소뼈','소잡뼈','우골','소갈비','소내장','소창자','우지','소기름','소안심','소등심', '사골'], 
                17: ['오징어'],
                18: ['조개류','굴','홍합','조개','백합','바지락','전복'], 
                19: ['잣'],
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }

# allergy_dict의 모든 값을 '|'로 연결하여 정규식 패턴을 만듭니다.
pattern = '|'.join(sum(allergy_dict.values(), []))

# df['rawmtrl']의 문자열에서 pattern과 일치하는 부분을 찾습니다.
# 결과는 True/False 값을 가지는 데이터프레임 형태로 반환됩니다.
matches = df_allergy['preprocessing'].str.contains(pattern, flags=re.IGNORECASE, regex=True)

# 결과를 저장할 빈 리스트를 만듭니다.
result = [[] for _ in range(len(df_allergy))]

# matches 데이터프레임의 각 행에 대해서
for i, row in matches.iteritems():
    # True인 경우
    if row:
        # 해당 행의 rawmtrl 문자열에서 pattern과 일치하는 모든 단어를 찾아서
        # 그 결과를 리스트로 만듭니다.
        words = re.findall(pattern, df_allergy.at[i, 'preprocessing'], flags=re.IGNORECASE)
        
        # 각 단어에 대해서
        for word in words:
            # allergy_dict에서 해당 단어를 포함하는 키를 찾아서
            for key, values in allergy_dict.items():
                if word.lower() in [v.lower() for v in values]:
                    # 해당 키를 result 리스트에 추가합니다.
                    result[i].append(key)

# 결과 리스트에서 중복된 값을 제거합니다.
result = [list(set(r)) for r in result]

# 결과를 새로운 칼럼으로 추가합니다.
df_allergy['allergens_num'] = result

<ipython-input-20-07ab9a820bca>:36: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in matches.iteritems():


In [21]:
# 결과를 저장할 리스트를 생성합니다.
result = []

# matches 데이터프레임의 각 행에 대해서
for i, row in matches.iteritems():
    # True인 경우
    if row:
        # 해당 행의 rawmtrl 문자열에서 pattern과 일치하는 모든 단어를 찾아서
        # 그 결과를 리스트로 만듭니다.
        words = re.findall(pattern, df_allergy.at[i, 'preprocessing'], flags=re.IGNORECASE)
        
        # 해당하는 인덱스에 1을 저장하는 리스트를 생성합니다.
        allergens = [0] * len(allergy_dict)
        
        # 각 단어에 대해서
        for word in words:
            # allergy_dict에서 해당 단어를 포함하는 키를 찾아서
            for key, values in allergy_dict.items():
                if word.lower() in [v.lower() for v in values]:
                    # 해당하는 인덱스에 1을 저장합니다.
                    allergens[key-1] = 1
        
        # 결과 리스트에 추가합니다.
        result.append(allergens)
    else:
        # False인 경우에는 모든 요소가 0인 리스트를 추가합니다.
        result.append([0] * len(allergy_dict))

# 결과를 새로운 칼럼으로 추가합니다.
df_allergy['allergens'] = result

<ipython-input-21-06367ea9fda1>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in matches.iteritems():


In [23]:
# 결과를 저장할 리스트를 생성합니다.
result = []

# matches 데이터프레임의 각 행에 대해서
for i, row in matches.iteritems():
    # True인 경우
    if row:
        # 해당 행의 rawmtrl 문자열에서 pattern과 일치하는 모든 단어를 찾아서
        # 그 결과를 리스트로 만듭니다.
        words = re.findall(pattern, df_allergy.at[i, 'preprocessing'], flags=re.IGNORECASE)
        
        # 해당하는 인덱스에 1을 저장하는 리스트를 생성합니다.
        allergens = [0] * (len(allergy_dict) + 1)  # 0을 추가하기 위해 리스트의 길이를 1 증가시킵니다.
        
        # 각 단어에 대해서
        for word in words:
            # allergy_dict에서 해당 단어를 포함하는 키를 찾아서
            for key, values in allergy_dict.items():
                if word.lower() in [v.lower() for v in values]:
                    # 해당하는 인덱스에 1을 저장합니다.
                    allergens[key] = 1  # 인덱스를 key로 변경하고, 1을 저장합니다.
        
        # 결과 리스트에 추가합니다.
        result.append(allergens)
    else:
        # False인 경우에는 모든 요소가 0인 리스트를 추가합니다.
        result.append([0] * (len(allergy_dict) + 1))  # 0을 추가하기 위해 리스트의 길이를 1 증가시킵니다.

# 결과를 새로운 컬럼으로 추가합니다.
for i in range(len(allergy_dict) + 1):
    column_name = f'allergen_{i}'
    df_allergy[column_name] = [row[i] for row in result]

<ipython-input-23-15d8b5255d2b>:37: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in matches.iteritems():


In [25]:
import numpy as np
df_allergy.columns
# df_allergy.loc[df_allergy['allergens'].apply(lambda x: x == []), 'allergens'] = np.array([[0]])

Index(['Unnamed: 0', 'rnum', 'rawmtrl', 'allergy', 'preprocessing',
       'allergens_num', 'allergens', 'allergen_0', 'allergen_1', 'allergen_2',
       'allergen_3', 'allergen_4', 'allergen_5', 'allergen_6', 'allergen_7',
       'allergen_8', 'allergen_9', 'allergen_10', 'allergen_11', 'allergen_12',
       'allergen_13', 'allergen_14', 'allergen_15', 'allergen_16',
       'allergen_17', 'allergen_18', 'allergen_19'],
      dtype='object')

In [45]:
df_allergy.head()

,Unnamed: 0,rnum,rawmtrl,allergy,preprocessing,allergens_num,allergens,allergen_0,allergen_1,allergen_2,...,allergen_10,allergen_11,allergen_12,allergen_13,allergen_14,allergen_15,allergen_16,allergen_17,allergen_18,allergen_19
0,0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"다시마, 김, 새우, 옥수수기름, 건표고버섯, 옥배유, 둥굴레, 대파, 감초, 정제...","[9, 18]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드,"다시마, 김, 새우, 아몬드, 옥수수기름, 건표고버섯, 옥배유, 둥굴레, 대파, 감...","[9, 18]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"다시마, 김, 새우, 옥수수기름, 건표고버섯, 옥배유, 둥굴레, 대파, 감초, 정제...","[9, 18]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,4,군고구마,없음,군고구마,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5,군고구마,없음,군고구마,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Target/Input data

In [26]:
import torch
data_set = df_allergy.drop(['Unnamed: 0', 'rnum', 'rawmtrl', 'allergy', 'allergens_num', 'allergens'], axis=1)
# data_set.rename(columns={'allergens1': 'label'}, inplace=True)
data_set.head()
y = np.array(data_set.iloc[:,1:])
y = torch.tensor(y, dtype=torch.float)

In [27]:
X = data_set['preprocessing']

# 토크나이징

In [28]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from sklearn.model_selection import train_test_split

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [29]:
train_data, test_data, train_label, test_label = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
max_len = 300 #97%

In [32]:
tokenized_train_sentences = tokenizer(
      list(train_data),
      return_tensors="pt",
      padding=True,
      truncation=True,
      add_special_tokens=True,
      max_length = max_len
)
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '말', '##차', ',', '갈', '##색', '##설', '##탕', ',', '버', '##터', ',', '우', '##유', ',', '밀', ',', '태', '##움', '##용', '##융', '##소', '##금', ',', '유', '##정', '##란', ',', '밀', '##가', '##루', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [33]:
tokenized_test_sentences = tokenizer(
      list(test_data),
      return_tensors="pt",
      padding=True,
      truncation=True,
      add_special_tokens=True,
      max_length = max_len
)
print(tokenized_test_sentences[0])
print(tokenized_test_sentences[0].tokens)
print(tokenized_test_sentences[0].ids)
print(tokenized_test_sentences[0].attention_mask)

Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '조', '##제', '##분', '##유', ',', '골', '##드', '##식', '##빵', '##가', '##루', ',', '옥', '##수', '##수', '##전', '##분', ',', '옥', '##수', '##수', ',', '정', '##제', '##소', '##금', ',', '밀', ',', '빵', '##효', '##모', ',', '등', '##심', ',', '밀', '##글', '##루', '##텐', ',', '산', '##서', '##피', '##로', '##인', '##산', '##나', '##트', '##륨', ',', '강', '##력', '##밀', '##가', '##루', ',', '마', '##가', '##린', ',', '세', '##르', '##비아', ',', '정', '##제', '##수', ',', '돼', '##지고', '##기', ',', '백', '##설', '##탕', ',', '메', '##타', '##인', '##산', '##나', '##트', '##륨', ',', '베', '##타', '##믹', '##스', ',', '가', '##공', '##유', '##지', ',', '산', '##도', '##조', '##절', '##제', ',', '박', '##력', '##분', ',', '폴', '##리', '##인', '##산', '##나', '##트', '##륨', ',', '생', '##이스', '##트', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [35]:
import torch
class SingleSentDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

      def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

      def __len__(self):
            return len(self.labels)


In [36]:
train_dataset = SingleSentDataset(tokenized_train_sentences, train_label)
test_dataset = SingleSentDataset(tokenized_test_sentences, test_label) #데이터셋 확정 

# 모델링

In [37]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=20,problem_type="multi_label_classification")

training_args = TrainingArguments(
    output_dir='/results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500, #러닝레이트 (0~맥시멈 갔다가 떨어트리는 식으로 구성)
    weight_decay=0.01,
    logging_dir='/logs',
    logging_steps=500,
    save_steps=500,
    save_total_limit=2
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [38]:
trainer = Trainer(
    model=model,               
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset = test_dataset
)

In [39]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-35-f76d4cb871eb>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-35-f76d4cb871eb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Step,Training Loss
500,0.250300
1000,0.050400
1500,0.023500


<ipython-input-35-f76d4cb871eb>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-35-f76d4cb871eb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-35-f76d4cb871eb>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-35-f76d4cb871eb>:9: UserWarning: To copy construct from a tensor, it is recommended to

TrainOutput(global_step=1890, training_loss=0.08922211505748608, metrics={'train_runtime': 3046.6156, 'train_samples_per_second': 19.814, 'train_steps_per_second': 0.62, 'total_flos': 9307772891604000.0, 'train_loss': 0.08922211505748608, 'epoch': 5.0})

In [40]:
torch.save(model.state_dict(), 'nlp_model.pth')

# 성능 확인

In [60]:
def sentences_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )
    tokenized_sent.to('cuda')

    with torch.no_grad():  # Disable gradient calculation
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
        )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()  # Apply sigmoid to convert logits to probabilities
    return probabilities



In [61]:
number = 10000 #<--
probabilities = sentences_predict(df_allergy['preprocessing'][number])
print(probabilities)
print(df_allergy['allergens'][number]) #라벨링 정답
print(df_allergy['preprocessing'][number]) #실제 성분


[[0.0053876  0.00862696 0.00762651 0.00762507 0.00493951 0.01104899
  0.01216713 0.00721323 0.01225878 0.8844594  0.00740734 0.00841823
  0.00557001 0.01324212 0.01616424 0.00606255 0.01659705 0.01592068
  0.01765066 0.0142982 ]]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
정제소금, 멸치액젓, 찹쌀풀, 불린고추, 새우액젓, 알파찹쌀분말, 고춧가루, 다시마육수, 양파, 멸치, 다시마, 홍고추, 생강, 건고추, 무, 마늘, 절임열무, 열무, 쪽파
